# 선도개발 - 구글 뉴스 피드 수집

**1. 구글 뉴스 피드 수집**

In [5]:
import re
from requests_html import HTMLSession

URL_GOOGLE_FINANCE_NEWEST_RSS = 'https://news.google.com/rss/topics/CAAqIQgKIhtDQkFTRGdvSUwyMHZNREpmTjNRU0FtdHZLQUFQAQ?hl=ko&gl=KR&ceid=KR%3Ako'

session = HTMLSession()
res = session.get(URL_GOOGLE_FINANCE_NEWEST_RSS)
res_text = res.text
normal_rss_xml = re.sub(r'\<\?xml[^\?].+\?\>', '', res_text)
print(normal_rss_xml)

<rss version="2.0" xmlns:media="http://search.yahoo.com/mrss/"><channel><generator>NFE/5.0</generator><title>금융 - 최신 뉴스 - Google 뉴스</title><link>https://news.google.com/topics/CAAqIQgKIhtDQkFTRGdvSUwyMHZNREpmTjNRU0FtdHZLQUFQAQ?hl=ko&amp;gl=KR&amp;ceid=KR:ko</link><language>ko</language><webMaster>news-webmaster@google.com</webMaster><copyright>2023 Google Inc.</copyright><lastBuildDate>Sun, 22 Oct 2023 09:15:29 GMT</lastBuildDate><description>Google 뉴스</description><item><title>[코인리더스] (종합) XRP 일주일새 7% 올라...`SEC 리플 경영진 소송 취하` 해석 엇갈려 - 코인리더스</title><link>https://news.google.com/rss/articles/CBMiH2h0dHBzOi8vbS5jb2lucmVhZGVycy5jb20vODUwMjLSAQA?oc=5</link><guid isPermaLink="false">2528866655</guid><pubDate>Sun, 22 Oct 2023 02:12:00 GMT</pubDate><description>&lt;ol&gt;&lt;li&gt;&lt;a href="https://news.google.com/rss/articles/CBMiH2h0dHBzOi8vbS5jb2lucmVhZGVycy5jb20vODUwMjLSAQA?oc=5" target="_blank"&gt;[코인리더스] (종합) XRP 일주일새 7% 올라...`SEC 리플 경영진 소송 취하` 해석 엇갈려&lt;/a&gt;&amp;nbsp;&amp;nbsp;&lt;f

**2. 뉴스 크롤링**

In [7]:
from pyquery import PyQuery as pq

def scrap_origin_news(item: str):
    item = pq(item)
    google_link = item.find('link').text()
    session = HTMLSession()
    res = session.get(google_link)
    original_news_link = res.html.pq.find('a').eq(0).attr('href')
    res = session.get(original_news_link)
    news_doc = res.html.pq
    original_content = news_doc.text()
    return (google_link, original_news_link, original_content)

doc = pq(normal_rss_xml)
items = doc.find('item')[0:1] # 테스트를 위해 하나만
news = [scrap_origin_news(item) for item in items]

print(news)

[('https://news.google.com/rss/articles/CBMiH2h0dHBzOi8vbS5jb2lucmVhZGVycy5jb20vODUwMjLSAQA?oc=5', 'https://m.coinreaders.com/85022', '[코인리더스] (종합) XRP 일주일새 7% 올라...`SEC 리플 경영진 소송 취하` 해석 엇갈려\n#logimg {height:40px;}\n//webview 에서 확대축소 가능하게 document.querySelector("meta[name=viewport]").setAttribute(\'content\', \'width=device-width, initial-scale=1.0, minimum-scale=1.0, maximum-scale=2.0, user-scalable=yes\'); document.querySelector("meta[name=viewport]").setAttribute(\'content\', \'width=device-width, initial-scale=1.0, minimum-scale=1.0, maximum-scale=2.0, user-scalable=yes\'); if (document.location.protocol == \'http:\') { document.location.href = document.location.href.replace(\'http:\', \'https:\'); } var kakao_key=\'\'; Kakao.init(kakao_key); function sendLink() { if(kakao_key){ Kakao.Link.cleanup(); Kakao.Link.sendScrap({ requestUrl: \'https://m.coinreaders.com/85022\', installTalk : true }); } else { alert(\'준비중입니다.\'); } } function sendLink2() { if(kakao_key){ Kakao.Link.cleanup